In [28]:
# Import python libraries
import numpy as np
import pandas as pd
import welleng as we
import plotly.graph_objects as go
from plotly.offline import plot

In [29]:
# Read the Well files
data = pd.read_csv("well1/well1_survey.txt",sep="\t")

In [30]:
data.head()

,# MD(mKB),Incl(deg),Azm(deg),Grav(g),Mag(Gs),DipA(deg),TVD(mKB),VS(m),NS(m),EW(m),DLS(deg/30m),SS(m),Status
0,0.00,0.00,0.00,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,796.93,OK
1,6.40,0.00,0.00,NaN,NaN,NaN,6.40,0.00,0.00,0.00,0.00,790.53,OK
2,33.00,0.00,0.00,NaN,NaN,NaN,33.00,0.00,0.00,0.00,0.00,763.93,OK
3,49.75,0.91,104.27,NaN,NaN,NaN,49.75,-0.04,-0.03,0.13,1.63,747.18,OK
4,77.70,0.75,117.99,NaN,NaN,NaN,77.70,-0.19,-0.17,0.51,0.27,719.23,OK


In [31]:
# Clean dataframe
columns_to_drop = ["Grav(g)","Mag(Gs)","DipA(deg)"]

data = data[data['# MD(mKB)'] != '# EOF']

In [32]:
data.head()

,# MD(mKB),Incl(deg),Azm(deg),Grav(g),Mag(Gs),DipA(deg),TVD(mKB),VS(m),NS(m),EW(m),DLS(deg/30m),SS(m),Status
0,0.00,0.00,0.00,NaN,NaN,NaN,0.00,0.00,0.00,0.00,0.00,796.93,OK
1,6.40,0.00,0.00,NaN,NaN,NaN,6.40,0.00,0.00,0.00,0.00,790.53,OK
2,33.00,0.00,0.00,NaN,NaN,NaN,33.00,0.00,0.00,0.00,0.00,763.93,OK
3,49.75,0.91,104.27,NaN,NaN,NaN,49.75,-0.04,-0.03,0.13,1.63,747.18,OK
4,77.70,0.75,117.99,NaN,NaN,NaN,77.70,-0.19,-0.17,0.51,0.27,719.23,OK


In [33]:


#read md, inc and azi data from dataframe
md_values = data['# MD(mKB)'].values
inc_values = data['Incl(deg)'].values
azi_values = data['Azm(deg)'].values

# Create a survey
survey_data = we.survey.Survey(md=md_values, inc=inc_values, azi=azi_values)


In [34]:

# Interpolate survey - generate points every 30 meters
survey_data_interp = survey_data.interpolate_survey(step=30)

# Create a 3D scatter plot
fig = go.Figure()

# Add interpolated survey data as a blue line
fig.add_trace(go.Scatter3d(
    x=survey_data_interp.x,
    y=survey_data_interp.y,
    z=survey_data_interp.z,
    mode='lines',
    line=dict(color='blue'),
    name='survey_interpolated'
))

# Add filtered survey data as red markers
fig.add_trace(go.Scatter3d(
    x=survey_data.x,
    y=survey_data.y,
    z=survey_data.z,
    mode='markers',
    marker=dict(color='red', size=3),  # Adjust marker size here
    name='survey'
))

# Reverse the z-axis for better visualization
fig.update_scenes(zaxis_autorange="reversed")

# Customize the layout
fig.update_layout(
    width=800,  # Set the width of the plot
    height=800,  # Set the height of the plot
    scene=dict(
        xaxis_title='X Axis Title',
        yaxis_title='Y Axis Title',
        zaxis_title='Z Axis Title'
    )
)

# Show the plot
fig.show()

In [43]:
ureg = we.units.ureg

node0 = we.node.Node(pos=[0, 0, 0], md=0, inc=0, azi=0)
node1 = we.node.Node(md=731.5, inc=0, azi=0)
node2 = we.node.Node(md=1280, inc=44, azi=0)
node3 = we.node.Node(md=3718, inc=44, azi=0)

# create an empty list
connectors = []

# append the first connector from the first to the second node
connectors.append(
    we.connector.Connector(node0, node1)
)

# us the newly created node in the connector list and connect it to the third node
connectors.append(
    we.connector.Connector(connectors[-1].node_end, node2, dls_design=2)
)

# similarly connector the current last node in the list to the final node
connectors.append(
    we.connector.Connector(connectors[-1].node_end, node3, dls_design=4)
)

# use the list of connectors as input to create a Survey instance
survey_example_2 = we.survey.from_connections(
    connectors
).interpolate_survey(step=30)

survey_example_2.figure().show()


In [55]:
import plotly.graph_objects as go

# Create a scatter plot for the survey points
fig = go.Figure()

# Add scatter points for the survey nodes
fig.add_trace(go.Scatter3d(
    x=survey_example_2.x,
    y=survey_example_2.y,
    z=survey_example_2.z,
    mode='markers',
    marker=dict(size=5, color='blue'),
    name='Survey Points'
))

# Reverse the z-axis for better visualization
fig.update_scenes(zaxis_autorange="reversed")

# Customize the layout
fig.update_layout(
    scene=dict(
        xaxis_title='East(+)/West(-)',
        yaxis_title='North(+)/South(-)',
        zaxis_title='TVD',
        aspectmode='cube',
        camera=dict(eye=dict(x=1.5, y=1.5, z=1.5))
    ),
    width=800,
    height=600,
    margin=dict(l=0, r=0, b=0, t=0)
)

# Show the plot
fig.show()


In [62]:
# initiate a BHA instance
bha = we.architecture.BHA(
    name='8 1/2" Drilling BHA', top=0, bottom=3718, method="bottom_up")

# add some Drill Collars
bha.add_section(
    od=(6.5 * ureg.inches).to('meters').m,
    id=(4 * ureg.inches).to('meters').m,
    length=(372 * ureg.ft).to('meters').m,
    unit_weight=(146.90 * ureg('lbs / ft').to('kg / meters')).m * 9.81,
    name='6 1/2" DC'
)

# add some HWDP
bha.add_section(
    od=(4.5 * ureg.inches).to('meters').m,
    tooljoint_od=(6.375 * ureg.inches).to('meters').m,
    id=(4 * ureg.inches).to('meters').m,
    length=(840 * ureg.ft).to('meters').m,
    unit_weight=(46.90 * ureg('lbs / ft').to('kg / meters')).m * 9.81,
    name='5" HWDP'
)

# add the drill pipe back to surface
bha.add_section(
    od=(5.0 * ureg.inches).to('meters').m,
    tooljoint_od=(6.625 * ureg.inches).to('meters').m,
    id=(3.625 * ureg.inches).to('meters').m,
    length=None,
    unit_weight=(20 * ureg('lbs / ft').to('kg / meters')).m * 9.81,
    name='5" DP'
)


In [72]:
# initiate a WellBore instance
wellbore = we.architecture.WellBore(
    name='8 1/2" Hole Section', top=0, bottom=3718, method='top_down'
)

# add the casing
wellbore.add_section(
    od=9+5/8, id=8.5,
    bottom=3688 * 0.7,
    unit_weight=(68 * ureg('lbs / ft').to('kg / meters')).m,
    coeff_friction_sliding=0.39,
    name='production 9 5/8" casing'
)

# add the open hole section below the casing to the section TD
wellbore.add_section(
    od=None, id=8.5, bottom=3718, unit_weight=None,
    coeff_friction_sliding=0.39, name='8 1/2" OH'
)



In [75]:
hookload = we.torque_drag.HookLoad(
    survey=survey_example_2, wellbore=wellbore, string=bha,
    fluid_density=11.6 / 8.33, step=30,
    name='8 1/2" Hole Section', ff_range=(0.1, 0.4, 0.1)
)

In [78]:
hookload.figure().show()

In [79]:
# Weight on Bit wob, Torque on Bottom/Bit tob and the overpull, all in Newtons

t_and_d = we.torque_drag.TorqueDrag(
    survey=survey_example_2, wellbore=wellbore, string=bha,
    fluid_density=9.8 / 8.33,
    wob=10_000, tob=(2_000 * 1.356), overpull=50_000,
    name='8 1/2" Hole Section'
)

t_and_d.figure().show()

DimensionalityError: Cannot convert from 'number_meter' ([length] / [mass]) to 'ft_lbf' ([length] ** 2 * [mass] / [time] ** 2)